# 03 - Baseline Model

## Setup

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown

sns.set(style="darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "Tic-Tac-Toe"
COLAB = 'google.colab' in sys.modules

DEBUG = False
SEED = 666

In [66]:
COLAB = 'google.colab' in sys.modules

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)
else:
  ROOT = "./"

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Load Dataset

In [67]:
df = pd.read_pickle(f"{ROOT}/data/data.pkl")
print(df.shape)
df.head()

(958, 10)


,Top-left-square,Top-middle-square,Top-right-square,Middle-left-square,Middle-middle-square,Middle-right-square,Bottom-left-square,Bottom-middle-square,Bottom-right-square,Score
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1


## Preprocessing Data

In [68]:
target = "Score"
print(f"target = {target}")

cat_features = [c for c in df.select_dtypes("category").columns if c!= target]
print(f"\nCategorical features ({len(cat_features)}): {cat_features}")


target = Score

Categorical features (9): ['Top-left-square', 'Top-middle-square', 'Top-right-square', 'Middle-left-square', 'Middle-middle-square', 'Middle-right-square', 'Bottom-left-square', 'Bottom-middle-square', 'Bottom-right-square']


In [69]:
X = df[cat_features]
y = df[target]

## Train/Test Split

In [70]:
y.value_counts(normalize=True)

Score
1     0.653445
-1    0.346555
Name: proportion, dtype: float64

In [71]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train,X_test,y_train,y_test = train_test_split(X,y, train_size=0.80, stratify=y, random_state=SEED)

print(X_train.shape,X_test.shape)
y.value_counts(normalize=True)

(766, 9) (192, 9)


Score
1     0.653445
-1    0.346555
Name: proportion, dtype: float64

## Eval Models

In [72]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import OneHotEncoder

#ohe = OneHotEncoder()
#ohe.fit(X_train)
#X_train = ohe.transform(X_train)
#X_test = ohe.transform(X_test)

classifiers = {
    "KNN" : KNeighborsClassifier(n_jobs=-1),
    "KNN(3)" : KNeighborsClassifier(3,n_jobs=-1),
    "DT" : DecisionTreeClassifier(max_features=9),
    "DT(max_depth=5)" : DecisionTreeClassifier(max_depth=5,max_features=9),
    "Perceptron" : Perceptron(n_jobs=-1),
}

In [73]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

metrics = {
    'accuracy' : accuracy_score,
    'precision' : precision_score,
    'recall' : recall_score,
    'f1' : f1_score,
    'roc_auc' : roc_auc_score,
}

In [74]:
def generate_metrics():
    
    data = []
    
    for model_name, model in classifiers.items():
        
        print (f"{model_name} ...")
        
        row = {'Model': model_name}
        model.fit(X_train, y_train)
        
        for metric_name, metric in metrics.items():
            # Scoring on SEEN data - effectively "useless"
            y_pred = model.predict(X_train)
            row['train_'+metric_name] = metric(y_train, y_pred)
        
            # Scoring on UNSEEN data - important
            y_pred = model.predict(X_test)
            row['test_'+metric_name] = metric(y_test, y_pred)
            
        data.append(row)
    return pd.DataFrame(data)

df_results = generate_metrics()
print(df_results.shape)

KNN ...


KNN(3) ...
DT ...
DT(max_depth=5) ...
Perceptron ...
(5, 11)


In [75]:
def highlight_col(x):
    model_color = 'background-color: lightgreen'
    alt_color = ['background-color: lightblue','background-color: lightyellow']
    
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)  
    df1.iloc[:, 0] = model_color
    for k in range (1,df.shape[1],2):
        df1.iloc[:,k:k+2] = alt_color[(k//2)%2] 
    return df1 
   
df_results.style.apply(highlight_col, axis=None)

,Model,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,train_roc_auc,test_roc_auc
0,KNN,0.998695,0.994792,0.998008,0.992063,1.000000,1.000000,0.999003,0.996016,0.998113,0.992537
1,KNN(3),1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,DT,1.000000,0.958333,1.000000,0.960630,1.000000,0.976000,1.000000,0.968254,1.000000,0.950687
3,DT(max_depth=5),0.916449,0.869792,0.937876,0.884615,0.934132,0.920000,0.936000,0.901961,0.908575,0.848060
4,Perceptron,0.983029,0.989583,0.974708,0.984252,1.000000,1.000000,0.987192,0.992063,0.975472,0.985075


KNN(3) já obtivemos 100% de precisão, accuracy e em outras métricas.... Não há necessidade de Hyperparameter Tunning, com gridSerach por exemplo...

## Save Best Model

### KNN(3)

In [76]:
import joblib

In [77]:
knn3_model = KNeighborsClassifier(3,n_jobs=-1)
knn3_model.fit(X_train, y_train)

KNeighborsClassifier(n_jobs=-1, n_neighbors=3)

In [78]:
joblib.dump(knn3_model,f"{ROOT}/output/tic-tac-toe-model-data.joblib")

['.//output/tic-tac-toe-model-data.joblib']

# Treino no Dataset Modificado

## Load Dataset

In [79]:
df = pd.read_pickle(f"{ROOT}/data/data_3.pkl")
print(df.shape)
df.head()

(5478, 10)


,Top-left-square,Top-middle-square,Top-right-square,Middle-left-square,Middle-middle-square,Middle-right-square,Bottom-left-square,Bottom-middle-square,Bottom-right-square,Score
0,1,-1,1,1,0,0,0,0,-1,-1
1,1,1,-1,0,-1,-1,1,-1,1,1
2,0,1,0,0,0,0,1,-1,0,-1
3,1,0,-1,1,0,0,0,0,0,1
4,-1,1,-1,-1,1,-1,1,0,1,1


## Preprocessing Data

In [80]:
target = "Score"
print(f"target = {target}")

cat_features = [c for c in df.select_dtypes("category").columns if c!= target]
print(f"\nCategorical features ({len(cat_features)}): {cat_features}")


target = Score

Categorical features (9): ['Top-left-square', 'Top-middle-square', 'Top-right-square', 'Middle-left-square', 'Middle-middle-square', 'Middle-right-square', 'Bottom-left-square', 'Bottom-middle-square', 'Bottom-right-square']


In [81]:
X = df[cat_features]
y = df[target]

## Train/Test Split

In [82]:
y.value_counts(normalize=True)

Score
1     0.535962
-1    0.464038
Name: proportion, dtype: float64

In [83]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train,X_test,y_train,y_test = train_test_split(X,y, train_size=0.80, test_size=0.20, stratify=y, random_state=SEED)

print(X_train.shape,X_test.shape)
y.value_counts(normalize=True)

(4382, 9) (1096, 9)


Score
1     0.535962
-1    0.464038
Name: proportion, dtype: float64

## Eval Models

In [84]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import OneHotEncoder

#ohe = OneHotEncoder()
#ohe.fit(X_train)
#X_train = ohe.transform(X_train)
#X_test = ohe.transform(X_test)

classifiers = {
    "KNN" : KNeighborsClassifier(n_jobs=-1),
    "KNN(3)" : KNeighborsClassifier(3,n_jobs=-1),
    "DT" : DecisionTreeClassifier(max_features=9),
    "DT(max_depth=5)" : DecisionTreeClassifier(max_depth=5,max_features=9),
    "Perceptron" : Perceptron(n_jobs=-1),
}

In [85]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

metrics = {
    'accuracy' : accuracy_score,
    'precision' : precision_score,
    'recall' : recall_score,
    'f1' : f1_score,
    'roc_auc' : roc_auc_score,
}

In [86]:
def generate_metrics():
    
    data = []
    
    for model_name, model in classifiers.items():
        
        print (f"{model_name} ...")
        
        row = {'Model': model_name}
        model.fit(X_train, y_train)
        
        for metric_name, metric in metrics.items():
            # Scoring on SEEN data - effectively "useless"
            y_pred = model.predict(X_train)
            row['train_'+metric_name] = metric(y_train, y_pred)
        
            # Scoring on UNSEEN data - important
            y_pred = model.predict(X_test)
            row['test_'+metric_name] = metric(y_test, y_pred)
            
        data.append(row)
    return pd.DataFrame(data)

df_results = generate_metrics()
print(df_results.shape)

KNN ...


KNN(3) ...
DT ...
DT(max_depth=5) ...
Perceptron ...
(5, 11)


In [87]:
def highlight_col(x):
    model_color = 'background-color: lightgreen'
    alt_color = ['background-color: lightblue','background-color: lightyellow']
    
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)  
    df1.iloc[:, 0] = model_color
    for k in range (1,df.shape[1],2):
        df1.iloc[:,k:k+2] = alt_color[(k//2)%2] 
    return df1 
   
df_results.style.apply(highlight_col, axis=None)

,Model,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,train_roc_auc,test_roc_auc
0,KNN,0.711091,0.607664,0.732903,0.634188,0.725415,0.632027,0.729140,0.633106,0.709978,0.605798
1,KNN(3),0.793930,0.613139,0.821333,0.646847,0.786718,0.611584,0.803653,0.628722,0.794490,0.613258
2,DT,1.000000,0.715328,1.000000,0.728027,1.000000,0.747871,1.000000,0.737815,1.000000,0.712835
3,DT(max_depth=5),0.671383,0.673358,0.685283,0.688633,0.715624,0.712095,0.700125,0.700168,0.667945,0.670390
4,Perceptron,0.619352,0.610401,0.660386,0.652091,0.596850,0.584327,0.627013,0.616352,0.621101,0.612399


## Hyperparameter Tunning

DecisionTreeClassifier

In [112]:
dt_model = DecisionTreeClassifier()

param_grid = {
    'max_depth': [7,11,12,15,16,20,None],
    'min_samples_split': [2, 5, 10, 0.1, 0.2, 0.5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [9],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train, )

grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 16,
 'max_features': 9,
 'min_samples_leaf': 1,
 'min_samples_split': 5}

In [113]:
best_dt = grid_search.best_estimator_

In [114]:
classifiers = {
    'best_DT' : best_dt,
}

In [117]:
df_results = generate_metrics()
df_results.style.apply(highlight_col, axis=None)

best_DT ...


,Model,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,train_roc_auc,test_roc_auc
0,best_DT,0.924920,0.729927,0.948092,0.758437,0.909749,0.727428,0.928525,0.742609,0.926099,0.730119


## Save Best Model

### KNN(3)

In [118]:
import joblib

In [93]:
#joblib.dump(knn3_model,f"{ROOT}/output/tic-tac-toe-model-data.joblib")

In [119]:
joblib.dump(best_dt,f"{ROOT}/output/tic-tac-toe-model-data2.joblib")

['.//output/tic-tac-toe-model-data2.joblib']